In [94]:
import os
PATH = './camara'
docs = []
files = os.listdir(PATH)
for f in files:
    with open(PATH + '/' + f, 'r', encoding='utf-8') as file:
        text = ' '.join(file.read().split('-')[2:])
        docs.append(text)

In [95]:
PATH = './gov'
files = os.listdir(PATH)
for f in files:
    with open(PATH + '/' + f, 'r', encoding='utf-8') as file:
        docs.append(file.read())

In [97]:
with open('data.txt', 'w', encoding='utf-8') as f:
    text = ''.join(docs)
    f.write(text)

In [54]:
# import re
# from functools import reduce

# def process_doc(doc):
#     paragraphs = doc.split('\n      ')
#     paragraphs[0] = ' '.join(paragraphs[0].split('\n')[1:])
#     paragraphs = [re.sub('\s+', ' ', par) for par in paragraphs]
#     paragraphs = [re.sub('^\s+', '', par) for par in paragraphs]
#     return paragraphs

# processed_docs = reduce(lambda x, y: x + y, [process_doc(doc) for doc in docs], [])

In [55]:
import nltk
# nltk.download('punkt')
processed_docs = []
for doc in docs:
    sents = nltk.sent_tokenize(doc)
    for sent in sents:
        if sent[0] not in ['(', ')'] and 'etc' not in sent.split(' '):
            processed_docs.append(sent)
print(processed_docs[-2])
print(len(processed_docs))

E Brasil acima de tudo e Deus acima de todos.
36942


In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
NUM_WORDS = 24766
NUM_WORDS = 10000

tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token='<unknown>')
tokenizer.fit_on_texts(processed_docs)
total_words = len(tokenizer.word_index) + 1
print(len(tokenizer.word_index))
print(tokenizer.word_index['a'])


24766
5


In [66]:
max_sequence_len = 16

sequences = []
k=0
for line in processed_docs:
    token_list = tokenizer.texts_to_sequences([line])[0]
#     if k == 5:
#         print(token_list)
#     k+=1
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
    sequences.append(n_gram_sequence)

In [67]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

sequences = pad_sequences(sequences, maxlen=max_sequence_len+1, padding='pre')

X = sequences[:, :-1]
labels = sequences[:, -1]
y = to_categorical(labels, num_classes=NUM_WORDS)


In [68]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.85)
# def shuffle(matrix, target, test_proportion):
#     ratio = int(matrix.shape[0]/test_proportion) #should be int
#     X_train = matrix[ratio:,:]
#     X_test =  matrix[:ratio,:]
#     Y_train = target[ratio:,:]
#     Y_test =  target[:ratio,:]
#     return X_train, X_test, Y_train, Y_test

# X_train, X_test, y_train, y_test = shuffle(X, y, 6)

In [60]:
import sys
print(sys.getsizeof(sequences) / 10e6, 'mb')

0.2512168 mb


In [69]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

model = Sequential()
model.add(Embedding(NUM_WORDS, 256, input_length=max_sequence_len))
# model.add(Bidirectional(LSTM(128, dtype='float', return_sequences=True)))
# model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, dtype='float')))
model.add(Dropout(0.2))
model.add(Dense(NUM_WORDS, activation='softmax', dtype='float'))

optimizer = Adam(lr=0.01)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 16, 256)           2560000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10000)             2570000   
Total params: 5,524,240
Trainable params: 5,524,240
Non-trainable params: 0
_________________________________________________________________
None


In [70]:
history = model.fit(X_train, y_train, epochs=30, batch_size=4096, 
                    validation_data=(X_valid, y_valid))

Train on 31400 samples, validate on 5542 samples
Epoch 1/30
31400/31400 [==============================] - 4s 136us/sample - loss: 8.1853 - accuracy: 0.0524 - val_loss: 7.4582 - val_accuracy: 0.0592
Epoch 2/30
31400/31400 [==============================] - 1s 47us/sample - loss: 7.2090 - accuracy: 0.0612 - val_loss: 7.3379 - val_accuracy: 0.0592
Epoch 3/30
31400/31400 [==============================] - 1s 47us/sample - loss: 6.9788 - accuracy: 0.0700 - val_loss: 7.1267 - val_accuracy: 0.0816
Epoch 4/30
31400/31400 [==============================] - 1s 47us/sample - loss: 6.7247 - accuracy: 0.0922 - val_loss: 6.9113 - val_accuracy: 0.1072
Epoch 5/30
31400/31400 [==============================] - 1s 46us/sample - loss: 6.4034 - accuracy: 0.1208 - val_loss: 6.6466 - val_accuracy: 0.1306
Epoch 6/30
31400/31400 [==============================] - 1s 47us/sample - loss: 5.9941 - accuracy: 0.1467 - val_loss: 6.3326 - val_accuracy: 0.1678
Epoch 7/30
31400/31400 [==============================] 

In [75]:
import numpy as np
seed_text = "dilma"
next_words = 100

index_to_word = {index: word for word, index in tokenizer.word_index.items()}

T = 0.9

for i in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')

    probas = model.predict(token_list, verbose=0)
    probas = np.array(probas[0][1:])
    probas = probas ** (1.0 / T)
    probas /= np.sum(probas)
    predicted = np.random.choice(range(1,NUM_WORDS), p=probas)
#     predicted = model.predict_classes(token_list, verbose=0)[0]
    try:
        while index_to_word[predicted] == seed_text[i-1] or index_to_word[predicted] in ['<unknown>', 'etc']:
            predicted = np.random.choice(range(1,NUM_WORDS), p=probas)
    except IndexError:
        pass
    seed_text += " " + (index_to_word[predicted] if predicted != 0 else '')
print(seed_text)

dilma rousseff sras pt al documento técnico material didático política salarial estabelecido projeto aqui também mais 7 políticos cuba homossexuais também vários gay hoje também dele aqui presentes armadas lamentavelmente merece presente momento sim mesmo instrumento parado marginais gay aqui novamente exa não vale nada não nada não é ele seja fraude não tenha resposta aqui não é fácil nominal grave ali mesmo contrário queimada golpe militar outubro anos depois também melhor presos dela embora transporte depois apresentei paredão entrar capitalismo dependentes medida provisória renda 20 oficial petróleo central aqui inocentes anunciado eu qualquer crime morrendo apenas mentira tinham lá sim
